# Otimizando o kernel da Transformada de Hough

##### Seções
- [Informações básicas](#Informações-básicas)
- [Parte A. Configuração](#Parte-A.-Configuração)
- [Parte B. Examinando a estrutura do código](#Parte-B.-Examinando-a-estrutura-do-código)
- [Parte C. Emular o código e examinar o relatório de otimização](#Parte-C.-Emular-o-código-e-examinar-o-relatório-de-otimização)
- [Parte D. Implementar memória local para os acumuladores](#Parte-D.-Implementar-memória-local-para-os-acumuladores)
- [Parte E. Desenrolar o loop interno e aplicar o atributo `ivdep`](#Parte-E.-Desenrolar-o-loop-interno-e-aplicar-o-atributo-ivdep)
- [Parte F. Usar bancos de memória para `accum_local`](#Parte-F.-Usar-bancos-de-memória-para-accum_local)

## Objetivos de aprendizado

* Compreender a Transformada de Hough
* Aprender como usar memória locar para melhorar o tempo de acesso aos dados
* Aprender como desenrolar loops para aumentar o paralelismo do código
* Aprender como usar bancos de memória para evitar gargalos no acesso

***
## Informações básicas

Neste exercício de laboratório, você estará otimizando um _kernel_ para um FPGA para calcular a __Transformada de Hough__ dos pixels em uma imagem. Você seguirá o fluxo apresentado em aula para obter essa otimização, verificará a funcionalidade usando emulação e usará um relatório de otimização HTML para decifrar quais otimizações podem ser benéficas.

A transformada de Hough é usada em aplicações de visão computacional. Depois que uma imagem foi processada com um algoritmo de detecção de bordas, como um filtro Sobel, você fica com uma imagem monocromática (preto/branco). É útil para muitos algoritmos de detecção adicionais considerar a imagem como um conjunto de linhas. No entanto, uma imagem de pixels em preto e branco não é uma representação conveniente ou útil dessas linhas para algoritmos como detecção de objeto. A __Transformada de Hough__ é uma transformação de pixels em um conjunto de "votos de linha".

Antes de entrar no código, aqui está a teoria por trás da Transformada de Hough.

É comumente conhecido que uma linha pode ser representada em uma forma de interceptação de declive:

$$y = mx + b$$

Nesta forma, cada linha pode ser representada por duas constantes únicas, a inclinação ($m$) e a interceptação y ($b$). Portanto, cada par ($m$, $b$) representa uma reta única. No entanto, esta forma apresenta alguns problemas. Primeiro, uma vez que as linhas verticais têm uma inclinação indefinida, não pode representar linhas verticais. Em segundo lugar, é difícil aplicar técnicas de limiarização.

Portanto, por razões computacionais em muitos algoritmos de detecção a forma normal de Hesse é usada. Esta forma possui a equação abaixo:

$$\rho = x~cos~\theta + y~cos~\theta$$

Nesta forma, cada linha única é representada por um par ($\rho$, $\theta$) (pronuncia-se “rô” e “téta”). Esta forma não tem nenhum problema em representar linhas verticais, e você aprenderá como o limiar pode ser facilmente aplicado depois que a Transformada de Hough for aplicada.

A figura a seguir mostra o que os valores de $\rho$ e $\theta$ representam na equação. Para cada linha que você deseja representar (veja a linha vermelha na imagem), haverá uma linha exclusiva que você pode desenhar da origem até ela com a distância mais curta (veja a linha cinza na imagem). Outra maneira de ver isso é a linha perpendicular à linha vermelha que cruza a origem. $\rho$ é a distância da linha mais curta que pode ser traçada da origem até a linha que você deseja representar. $\theta$ é o ângulo do eixo $x$ para essa linha.

<center><img src="Assets/Hesse.png"></center>

Ao trabalhar com uma imagem, um canto da imagem é tradicionalmente considerado como a origem (a origem não está no centro), então o maior valor que $\rho$ pode ser é a medida da diagonal da imagem. Você pode escolher que os valores de $\rho$ sejam todos positivos ou que possam ser positivos e negativos. Se você escolher que todos os valores de $\rho$ sejam positivos, o intervalo de $\theta$ vai de 0 a 360 graus. Se você escolher que $\rho$ possa ser positivo ou negativo, o intervalo de $\theta$ é de 0 a 180 graus. Esses intervalos são quantizados a fim de definir um espaço de solução finito.

Lembre-se de que a imagem antes de ser inserida na Transformada de Hough já passou por um algoritmo de detecção de bordas e, portanto, é monocromática (cada pixel é preto ou branco). As arestas detectadas são representadas por pixels brancos. A Transformada de Hough transformará os pixels brancos em uma matriz de votos para linhas.

Cada pixel branco na imagem é potencialmente um ponto em um conjunto de linhas. A imagem abaixo representa as linhas das quais um pixel pode potencialmente fazer parte. (Nota: nem todas as linhas potenciais são desenhadas, isso serve apenas para fins ilustrativos.) Uma linha com cada inclinação potencial que passa por aquele pixel é potencialmente uma linha que aparece na imagem. Portanto, um voto será acumulado para cada uma dessas linhas.

<center><img src="Assets/Lines.png"></center>

O código percorrerá cada pixel da imagem, acumulando votos nas linhas conforme avança. Conforme uma linha acumula mais votos, a probabilidade de ser uma representação correta para uma linha na imagem aumenta. Assim, conforme visualizado a seguir, a linha verde acumulará três votos, o que a tornará um candidato mais provável do que as demais linhas. Um limite pode ser facilmente aplicado, portanto, simplesmente definindo a quantidade de votos que é "suficiente" para definir se uma linha está presente ou não.

<center><img src="Assets/Lines3.png"></center>

Agora, vamos dar uma olhada no código para implementar essa transformação. Primeiro, o algoritmo completo será mostrado e, em seguida, explicado parte por parte.

```cpp
// Uma tabela de valores de sin e cos para os graus inteiros
#include "sin_cos_values.h"

char pixel_array[IMAGE_HEIGHT*IMAGE_WIDTH]; 
short accumulators[THETAS*RHOS*2];

for (uint y=0; y<IMAGE_HEIGHT; y++) { 
    for (uint x=0; x<IMAGE_WIDTH; x++) {
        unsigned short int increment = 0;
        if (pixel_array[(WIDTH*y)+x] != 0) { 
            increment = 1;
        } else {
            increment = 0;
        }
        for (int theta=0; theta<THETAS; theta++) {
            int rho = x*cos_table[theta] + y*sin_table[theta]; 
            accumulators[(THETAS*(rho+RHOS))+theta] += increment;
        }
    }
}
```

Vamos primeiro dar uma olhada nas declarações de array no topo do código. 

```cpp
char pixel_array[IMAGE_HEIGHT*IMAGE_WIDTH]; 
short accumulators[THETAS*RHOS*2];
```

A matriz de pixels é a própria imagem e cada pixel ocupa um lugar na matriz.

A matriz de acumuladores acompanhará nossos votos de linha. Cada lugar na matriz representa uma linha potencial na imagem. Lembre-se de que uma linha única é representada por um par ($\rho$, $\theta$). Portanto, o número de todas as linhas potenciais em nossa imagem é igual a todos os valores potenciais de $\rho$ vezes todos os valores potenciais de $\theta$. $\rho$ é a distância da origem, que é definida como um canto da imagem. O maior valor de $\rho$ é a medida da diagonal da imagem. Também deixaremos $\rho$ ser positivo ou negativo, de modo que $\theta$ seja limitado entre 0 e 180 graus. Quantizaremos em valores inteiros para $\rho$ e graus inteiros para $\theta$. Nosso número de acumuladores, portanto, é a medida da diagonal da imagem (RHOS no código) vezes 2 vezes 180 graus (THETAS no código).

Agora, vamos examinar o código para implementar o algoritmo.

```cpp
for (uint y=0; y<IMAGE_HEIGHT; y++) { 
    for (uint x=0; x<IMAGE_WIDTH; x++){
        ... 
    } 
}
```

O loop externo percorrerá cada pixel da imagem, acumulando votos para todas as linhas potenciais das quais um pixel poderia fazer parte.

```cpp
        unsigned short int increment = 0;
        if (pixel_array[(WIDTH*y)+x] != 0) { 
            increment = 1;
        } else {
            increment = 0;
        }
```

Se o pixel for branco (! = 0), então adicionaremos 1 a todos os acumuladores para linhas potenciais definidas por aquele pixel. Caso contrário, não adicionaremos nada ao acumulador. Fazemos isso desta maneira para que a lógica de controle dentro do FPGA e a lógica de computação que iremos duplicar posteriormente com pragmas sejam mais simples e consuma menos recursos lógicos.

```cpp
        for (int theta=0; theta<THETAS; theta++) {
            int rho = x*cos_table[theta] + y*sin_table[theta]; 
            accumulators[(THETAS*(rho+RHOS))+theta] += increment;
        }
```

Para cada localização de pixel, todas as linhas que podem ter essa localização de pixel como parte de seus valores precisam receber um voto. Lembre-se da fórmula que usamos para representar uma linha:

$$\rho = x~cos~\theta + y~cos~\theta$$

Os valores $x$ e $y$ são constantes durante este loop interno. Inseriremos todos os valores possíveis de $\theta$, junto com $x$ e $y$, e resolveremos para $\rho$ dado aquele $\theta$. Vamos então votar (adicionar 1 à localização do acumulador) para esse par ($\rho$, $\theta$). Desse modo, votamos em cada linha que é uma possibilidade, percorrendo as possibilidades em um arco de 0 a 180 graus.

Agora, vamos começar o laboratório e ver quais otimizações podemos fazer para melhorar o tempo total de execução no FPGA. Não se preocupe se você não entender completamente o algoritmo, é suficiente pensar nele como um pedaço de código conveniente para realizar otimizações. No entanto, se você quiser saber mais, a entrada da Wikipedia para a Transformada de Hough é um ótimo lugar para começar: https://en.wikipedia.org/wiki/Hough_transform

## Parte A. Configuração

Para este laboratório, você trabalhará diretamente no terminal do Jupyter e também abrirá os arquivos de código-fonte diretamente. Para começar, abra um terminal dentro do Jupyter, se ainda não tiver um aberto. Para abrir um terminal, clique duas vezes no botão `Terminal` na guia `Launcher` do Jupyter, como fizemos anteriormente. Se você não vir a guia `Launcher`, clique no botão “+” no canto superior esquerdo do Jupyter, e uma guia `Launcher` será aberta.

Aqui estão algumas notas importantes sobre o laboratório que você deve ler antes de prosseguir:

Este laboratório foi projetado para fornecer informações valiosas e experimentar cada etapa do caminho. Você passará por uma série de otimizações para um kernel. __Se você quiser pular as modificações de código e usar as soluções, está tudo bem! Você ainda aprenderá muito. Se você não chegar a todas as partes, tudo bem! Você ainda aprenderá muito.__ 

Soluções para cada etapa de codificação estão disponíveis no diretório `~/sycl-wscad-2022/labs/lab3/solutions`.

O arquivo `~/sycl-wscad-2022/labs/lab3/compile.sh` contém os comandos que você executará em cada etapa, você pode invocá-lo digitando `../compile.sh` dentro da respectiva pasta.

Um _script_ de limpeza está contido em cada subdiretório, se necessário. Execute-o digitando `source ./clean.sh`

## Parte B. Examinando a estrutura do código

Depois de abrir um terminal no Jupyter, no prompt do terminal, navegue até o diretório denominado `original` no `lab3`. O restante das instruções neste laboratório presumirá que você clonou o repositório do curso em seu diretório doméstico (indicado por `~`). Se você descompactou os arquivos em um diretório diferente, substitua o `~` nos comandos pelo diretório a partir do qual você iniciou.

```bash
$ cd ~/sycl-wscad-2022/labs/lab3/original
```

Abra o código-fonte dentro do Jupyter clicando duas vezes no arquivo `~/sycl-wscad-2022/labs/lab3/hough_transform.cpp` no navegador de arquivos no lado esquerdo do ambiente do Jupyter, conforme a figura abaixo.

<center><img src="Assets/Files.png"></center>

O código agora deve ser aberto em uma guia dentro do Jupyter, conforme mostrado abaixo.

<center><img src="Assets/Source.png"></center>

No editor de texto, pesquise __"Block off this code"__ usando `Ctrl-F` para abrir uma caixa de diálogo de busca. Isso o levará para a seção de código com todo o código SYCL. Bloquear o código dessa maneira e restringir todas as construções SYCL ao bloco garante que, quando o bloco terminar a execução, os objetos SYCL serão destruídos. As rotinas do destruidor para os objetos SYCL garantem que todo o trabalho com eles seja concluído antes da destruição. Dessa maneira, o bloco (entre `{}`) atua como um mecanismo de sincronização.

Isso é importante porque, sem esse mecanismo de sincronização, o _buffer_ que contém os acumuladores não será gravado de volta no host antes que o host leia a memória alocada para os acumuladores.

O bloqueio desta maneira é a técnica que a maioria dos exemplos SYCL usam para garantir a sincronização de dados no host.

Observe que outra maneira de sincronizar isso sem um bloco seria criar um acesso ao _buffer_ do host. Uma vez que apenas uma coisa pode interagir com o _buffer_ de cada vez, as interações do dispositivo terminariam antes que o host pudesse usar seu acessor. Se você gostaria de ver um exemplo dessa técnica alternativa, consulte o exemplo chamado __"FPGA tutorial: Caching local memory to improve performance"__ que você pode gerar usando o comando `oneapi-cli`.


O escopo de comandos em um programa SYCL é onde as ações são enviadas para a fila. Pesquise __"Device queue submit"__ no código-fonte. Aqui você verá onde começa o escopo de comandos. A captura de tela mostrada a seguir ilustra todo o escopo do comando.

<center><img src="Assets/Scope.png"></center>

Lembre-se de que no SYCL o escopo do _kernel_ é o que encerra o código do kernel que será executado no dispositivo. Pesquise no código por __"Call the kernel"__. É o código mostrado abaixo que compreende o código do kernel. Estaremos otimizando o código dentro do escopo do kernel durante este laboratório.

<center><img src="Assets/Kernel.png"></center>

Examine o restante do código para o nível de compreensão desejado. Se houver algo que você não entenda, seu instrutor ficará feliz em responder suas perguntas.

## Parte C. Emular o código e examinar o relatório de otimização

Você ainda deve estar no prompt do terminal no diretório `~/sycl-wscad-2022/labs/lab3/original`. Certifique-se de que você está neste diretório.

Agora iremos compilar o código para emulação. Lembre-se da apresentação e do laboratório anterior que a emulação é usada para garantir a funcionalidade do código, incluindo o código dentro do escopo do kernel. Para compilar para emulação, execute o seguinte comando:

```bash
dpcpp -fintelfpga -DFPGA_EMULATOR hough_transform.cpp -o fpga.emu
```

Este comando produzirá um arquivo chamado `fpga.emu`, que é um executável que será executado no host que contém o código do host, bem como uma versão emulada do código do kernel. Execute este código agora digitando o seguinte comando:

```bash
./fpga.emu
```

Você deve ver a seguinte saída em sua tela. Isso significa que a execução foi bem-sucedida. Se você estivesse trabalhando com seu próprio código, seria de se esperar que passasse por muitas rodadas de emulação para obter a funcionalidade correta do código.

<center><img src="Assets/Output.png"></center>

Depois de obter emulação bem-sucedida de seu _kernel_, a próxima etapa é examinar o relatório de otimização para examinar o desempenho estimado dos loops, a estrutura da memória do _kernel_ e a quantidade de recursos que o _kernel_ consome no FPGA. Faremos isso agora. Para compilar o código em um arquivo de objeto e gerar um relatório de otimização HTML estático para o _kernel_, execute os 2 comandos a seguir. O segundo desses comandos demorará um pouco mais para ser concluído do que o anterior (15 a 30 segundos).

```bash
dpcpp -fintelfpga -c hough_transform.cpp -o fpga.o 
dpcpp -fintelfpga -fsycl-link -Xshardware fpga.o
```

Após a conclusão dos comandos da última etapa, um subdiretório chamado `fpga.prj` será criado. Nesse diretório, há um subdiretório `reports`, e o relatório de otimização é chamado `report.html`. Abra esse relatório agora navegando até ele no painel esquerdo do Jupyter  e clicando duas vezes nele. Clique em __Trust HTML__ se necessário para que o arquivo de relatório seja totalmente aberto ou transfira-o para o seu computador.

Agora você verá o relatório de otimização de HTML estático em seu navegador. Observe as diferentes seções do relatório clicando nas caixas destacadas em azul na parte superior para obter uma visão geral do tipo de informação contida nelas. Esta primeira implementação da Transformada de Hough não foi muito boa. Vá para a seção `Loops Analysis` do relatório clicando na caixa `Throughput Analysis`, clicando nela e em `Loops Analysis`.

<center><img src="Assets/Report.png"></center>

Esta seção do relatório fornece uma análise do desempenho de seus loops. Para _kernels_ de item de trabalho único (aqueles lançados com a chamada API SYCL `single_task`), o Intervalo de Iniciação (II) de cada loop é calculado e relatado. Lembre-se de que II é o número de ciclos de clock entre as novas partes dos dados que estão sendo inseridas no pipeline de processamento. Um II alto significa que muitos ciclos são gastos paralisando, com o hardware não sendo usado.
Examine os valores II nesta versão do kernel clicando na linha que começa com Kernel: na seção `Loop List` do relatório.

O II mais alto tem 338 ciclos de clock! Isso é muito alto, chegando a centenas de ciclos de clock desperdiçados entre cada iteração do loop!
Observação: os nomes dos blocos no arquivo de relatório parecem estranhos. Isso ocorre porque o kernel foi escrito in-line no código usando funções lambda em vez de ser uma função própria. Os kernels de nomenclatura codificados dessa maneira no relatório podem ser mais legíveis por humanos em versões futuras da ferramenta.

Clique na linha do relatório onde o loop II mais alto é mostrado. Detalhes sobre por que este II é tão longo serão mostrados no painel inferior do relatório. O bloco de código onde o gargalo é inferido também será destacado. Você também pode pular para a linha de código onde o loop foi escrito clicando no link na coluna __Source Location__.

Os gargalos que ocorrem são __dependências de memória__. Isso significa que estamos aguardando a conclusão de uma operação da memória antes de iniciar uma nova iteração do pipeline. (Como uma observação lateral, o outro tipo de dependência que pode causar um gargalo é uma dependência de dados, o que significa que um cálculo leva muito tempo para ser concluído.) Todos eles ocorrem nas linhas 107 e 108, onde estamos obtendo valores do seno e tabelas de pesquisa de cosseno e procurando e incrementando acumuladores.

Examine os detalhes mostrados na parte inferior para este loop. Observe que as dependências de memória são mencionadas com frequência. As dependências de memória estão tornando o II do loop muito grande. Isso significa que estamos aguardando a conclusão de uma operação da memória antes de iniciar uma nova iteração do pipeline.

<center><img src="Assets/Details.png"></center>

Role até ver a seção intitulada __Most critical loop feedback path during scheduling__.

<center><img src="Assets/Critical.png"></center>

Uma das linhas de código que tem um grande impacto na programação do loop está acontecendo na linha 108. Quando o código é examinado, pode-se ver que esta linha acessa os acumuladores com uma carga e um armazenamento (para incrementar o valor). Vamos otimizar esse gargalo na próxima seção do laboratório.

Abra a seção `Area Analysis of System` do relatório. Expanda a seção `Kernel System`. Mostra os recursos usados pelo kernel.

<center><img src="Assets/Area.png"></center>

A próxima etapa seria compilar o kernel em um executável completo para o FPGA e executá-lo no próprio FPGA (incluindo o flag `-Xsprofile` se desejar ver informações de perfilamento no Intel&copy; VTune&trade; Amplifier). Lembre-se de que você pode fazer tudo isso (incluindo executá-lo em uma placa FPGA!) na DevCloud. Como essa etapa leva horas, não a faremos aqui. Para começar a usar o Intel DevCloud com FPGAs Intel, visite este site após o laboratório: https://software.intel.com/content/www/br/pt/develop/articles/getting-started-with-intel-devcloud-for-oneapi-projects.html e clique em __FPGA Vector-Add Sample Walkthrough__ do lado esquerdo da página.

Se você executasse o kernel em um FPGA no DevCloud, você descobriria que o tempo de execução é de cerca de __2,87038 segundos__. Iremos comparar isso com outras execuções enquanto otimizamos o kernel.

## Parte D. Implementar memória local para os acumuladores

Mude o diretório para `~/sycl-wscad-2022/labs/lab3/local_memory` digitando o seguinte comando no prompt do terminal.

```bash
cd ../local_memory
````

Abra o arquivo `~/sycl-wscad-2022/labs/lab3/hough_transform_CHANGEME.cpp` navegando até ele no painel esquerdo do Jupyter e clicando nele.

Para esta otimização, você implementará uma memória local para manter os valores do acumulador, já que nosso II ainda é muito alto devido ao tempo de acesso necessário para carregar e armazenar esses valores de/para a memória global. Lembre-se da apresentação de que, para implementar uma memória local em um único kernel de item de trabalho, você simplesmente declara um array dentro do escopo do kernel. No arquivo `hough_transform_CHANGEME.cpp`, crie uma memória local para os acumuladores, declarando uma matriz chamada `accum_local` no escopo do kernel do código. Ela deve ser do mesmo tamanho e tipo que a matriz chamada acumuladores.

Quando terminar de modificar o código, salve o arquivo como `hough_transform.cpp`. Faça isso usando a caixa de diálogo `File > Save File As...` dentro do ambiente do Jupyter.

__Lembre-se de que as soluções estão disponíveis se você precisar delas ou se quiser passar pelo laboratório mais rápido. A solução para esta etapa de recodificação está disponível em `~/sycl-wscad-2022/labs/lab3/solutions/local_memory/hough_transform.cpp.__

Compile o código para emulação usando o mesmo comando que você usou nas etapas anteriores. Se houver erros de sintaxe, corrija-os e recompile.

Execute o executável de emulação usando o comando `./fpga.emu`. Se você não vir a mensagem `VERIFICATION PASSED!`, corrija seu código e tente novamente.

Compile seu código em um arquivo de objeto e gere um relatório de otimização estática usando o método de 2 etapas com os comandos `dpcpp` usados nas etapas anteriores.

Abra o relatório de otimização estática navegando até ele no Jupyter e clicando duas vezes. Ele estará no seguinte local. Pode ser necessário clicar em __Trust HTML__.

`~/sycl-wscad-2022/labs/lab3/local_memory/fpga.prj/reports/report.html`

Abra a seção `Loops Analysis` do relatório e observe o II aprimorado. Nossa, que diferença! O maior II agora tem apenas ~2 ciclos de clock (é aproximado porque é uma interação com a memória global, que possui alguns de forma imprevisível).

Abra a seção `Area Analysis of System` do relatório. Abra também essa seção para a última compilação (do subdiretório `constant_cache`). Observe como a utilização de recursos do FPGA mudou, especialmente que as `onchip RAMs`  usadas aumentaram. Isso faz sentido porque as usamos para armazenar cópias locais dos acumuladores.

Não rodaremos no FPGA por questão de tempo, mas se você compilou e executasse este código no FPGA, você descobriria que o tempo de execução seria de cerca de __0,061376 segundos__, muito menor do que nosso último tempo de execução.

## Parte E. Desenrolar o loop interno e aplicar o atributo `ivdep`

Mude o diretório para `~/sycl-wscad-2022/labs/lab3/unroll_ivdep` digitando o seguinte comando no prompt do terminal.

```bash
cd ../unroll
```

Para essa otimização, você desenrolará o loop interno do código para direcionar o compilador para criar hardware de forma que mais iterações de loop possam ocorrer em paralelo. Você também aplicará o pragma ivdep ao loop para que o compilador saiba que as operações de memória dentro do loop são independentes das operações de memória que acontecem durante outras iterações do loop (se forem consideradas dependentes, o loop não será desenrolado).

Para desenrolar o loop, um pragma apresentado durante a aula precisa ser aplicado ao loop inserindo o pragma antes do loop no código. Além disso, para aplicar o atributo ivdep ao código, o atributo precisa ser inserido antes do loop. O pragma e o atributo devem ser aplicados ao loop que percorre todos os valores possíveis de theta. __Desenrole o loop 32 vezes, qualquer coisa maior resultará em longos tempos de compilação (o que estaria ok se não tivéssemos tempo limitado para o laboratório) para o estágio de relatório de otimização.__

Faça a alteração no código e salve-o como `hough_transform.cpp` ou copie na solução em `~/sycl-wscad-2022/labs/lab3/unroll/hough_transform.cpp`.

Compile o código para emulação e execute o executável de emulação (consulte as etapas anteriores para os comandos). Faça isso até ver a mensagem __VERIFICATION PASSED!__. 

Compile o código em um arquivo de objeto e relatório de otimização estática usando os comandos `dpcpp` apresentados nas etapas anteriores. Espere que esta etapa leve alguns minutos.

Abra o relatório de otimização estática navegando até ele e clicando, como nas etapas anteriores. Abra a seção `Loops Analysis` do relatório,observe que o loop foi desenrolado 32 vezes e que o II foi melhorado.

Depois, abra a seção `Kernel Memory Viewer` do relatório. Ela está sob o menu `Systems Viewers`. Clique em `accum_local` na lista `Kernel Memory List`, conforme mostrado abaixo. Esta seção do relatório está nos dando uma representação visual das estruturas de memória _onchip_ construídas para nosso código de escopo do _kernel_. O vermelho em geral é ruim, significa que há potencial estagnação nesses pontos de carregamento e armazenamento. Uma vez que desenrolamos o laço, 32 valores da estrutura de memória local acumulada são exigidos pela estrutura do laço desenrolado de uma vez. Essa demanda massiva da estrutura de memória causou a necessidade de arbitragem e introduziu potencial paralisação.

<center><img src="Assets/Memory.png"></center>

Se você compilasse em um executável FPGA completo e executasse esta versão do código em um FPGA, veria que o tempo de execução é de cerca de __0,018581 segundos__.

## Parte F. Usar bancos de memória para `accum_local`

Mude o diretório para `~/sycl-wscad-2022/labs/lab3/banking` digitando o seguinte comando no prompt do terminal.

```bash
cd ../banking
````

Para esta etapa, foram necessárias alterações mais extensas no código. Então, a recodificação foi feita para você.

A próxima otimização usará o atributo `numbanks`. Lembre-se de que os bancos são estruturas que possuem portas independentes do resto da estrutura da memória, mas que contêm apenas uma parte do conteúdo. Por exemplo, se criarmos 2 bancos, 1 banco conteria metade dos dados e o outro banco conteria a outra metade dos dados, cada metade poderia ser lida independentemente.

Os bancos serão criados usando o índice mais baixo e o atributo `numbanks` deve ser definido como uma potência de 2.

Duas alterações foram feitas no código desta etapa:
* Estruture `acum_local` como uma matriz bidimensional em vez de uma matriz unidimensional.
    * A dimensão inferior deve ser uma potência de 2 mais próxima de 180, então 256
    * Onde quer que você percorra os índices para `acum_local` precisará de uma mudança 
* Aplique o atributo `numbanks` à variável `acum_local`.

Abra o arquivo `hough_transform.cpp` dentro do ambiente do Jupyter. Observe que o banco foi declarado usando um atributo na linha 99. Observe também que a memória precisava ser tornada bidimensional para fazer isso, então isso teve implicações para outras partes do código no kernel.

```cpp
      [[intelfpga::numbanks(256)]]
      short accum_local[RHOS*2][256];
```

Compile o código para emulação e execute o executável de emulação até ver a mensagem __VERIFICATION PASSED!__. Compile o código para um arquivo de objeto e um relatório de otimização estática. Isto leverá alguns minutos.

Abra o relatório de otimização. Navegue até a seção `Memory Viewer` do relatório. Observe que __o vermelho se foi!__ Isso significa que não há mais travamento potencial ao acessar a estrutura da memória.

<center><img src="Assets/Memory2.png"></center>

O tempo de execução desta versão final do kernel é de cerca de __0,005577 segundos__ em um FPGA no Intel DevCloud.

## __Você chegou ao final do exercício e ao final do minicurso. Muito obrigado por participar!__

## Referências
- https://www.oneapi.io/
- https://www.intel.com/content/www/us/en/develop/documentation/oneapi-fpga-optimization-guide/
